# #405 Prestressed Tee beam
<i>Multispan tee beam from example #60 extended to demonstrate addition of prestressing tendons. 2 Approaches to prestressing are shown</i>
***

In [ ]:
import math

In [ ]:
# Input Params
span_lengths = [8, 15, 8] #m
tee_section_width = 1.4
tee_section_depth = 1.0
tee_section_thk_flange = 0.2
tee_section_thk_web = 0.3

loading = 50

# Tendon profile variables
minz = -tee_section_depth*0.6
maxz =  tee_section_depth*0.2

Ec_shrt = 35e6
phi = 2

approx_stress_at_transfer = 5e3 #kpa
approx_change_in_stress_after_transfer = -1.5e3 #kpa

# Tendon details
strand_diameter = 12 # mm
no_strands      = 19
tendon_area     = no_strands * math.pi * strand_diameter**2 * 0.25

# Tendon is defined by an equivalent diameter specified in mm
equiv_diameter  = math.sqrt(4*tendon_area/math.pi)
# And the tendon yield stress is specified in MPa
tendon_yield_stress = 1800 # 

# Dictionary of tendon properties used to create attribute
tendon_properties_dictionary = {"elasticShort":0, "shortAvg":-1.0, "diameter":equiv_diameter, "modulus":195.0E6, "wobbleFactor":0.01,
                                 "friction":0.19, "lossType":0, "tensileS":tendon_yield_stress, "relaxClass":2, "loss":2.5}

# Force applied
tendon_force = 0.7 * tendon_yield_stress*1e3 * tendon_area*1e-6 # kN

time_at_end_of_service = 365*100 # 100 years

#### Connect to LUSAS and create a new model database

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI import *
lusas = get_lusas_modeller()

from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)

if lusas.existsDatabase():
    raise Exception("This script will create a new model. Please save and close the current model and try again")

# Create a new model
lusas.newProject("Structural", "Prestress Tee Beam.mdl")
# Get the database
db = lusas.getDatabase()
# Set the analysis category & vertical axis
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")
db.setTimescaleUnits("Days") # specify age of concrete in days

# Create model attributes

In [ ]:
''' Create a mesh attribute'''
# Beam mesh with BMI21 (2 Noded Linear) Elements at 1m lengths
mesh_attr = db.createMeshLine("Beam Mesh").setSize("BMI21", 1)

In [ ]:
''' Create a geometric attribute'''
# Create a parametric section utility
util = db.createParametricSection('Tee Section Utility')
util.setType("T")
util.setDimensions(['B', 'D', 'tf', 'tw', 'r'], [tee_section_width, tee_section_depth, tee_section_thk_flange, tee_section_thk_web, 0.0])

# Now create the attribute which is based on the utility. The attribute provides additional transformations such as eccentricity and tapering
geom_section_attr = db.createGeometricLine('Tee Section')
geom_section_attr.setFromLibrary("Utilities", "", util.getName(), 0, 0, 0)

In [ ]:
def add_rebar(attr:IFReinforcementSection, row:int, faceIndex:int, layerIndex:int, barsCount:int, start:int, end:int, gap:float, dia:float):
    attr.setReinforcementValue("rebar", row, "faceIndex", faceIndex)
    attr.setReinforcementValue("rebar", row, "layerIndex", layerIndex)
    attr.setReinforcementValue("rebar", row, "barsCount", barsCount)
    attr.setReinforcementValue("rebar", row, "start", start)
    attr.setReinforcementValue("rebar", row, "end", end)
    attr.setReinforcementValue("rebar", row, "gap", gap)
    attr.setReinforcementValue("rebar", row, "barsDiameter", dia)
    attr.setReinforcementValue("rebar", row, "altBarsDiameter", -1.0)
    attr.setReinforcementValue("rebar", row, "endBarsDiameter", -1.0)
    attr.setReinforcementValue("rebar", row, "materialRef", "R1")

In [ ]:
''' Create a Reinforcement attribute'''
attr = db.createReinforcementSection("RnfSct1")
attr.setGeometricAttribute(geom_section_attr)
#attr.setValue("calculateCrackWidths", True)
attr.setReinforcementValue("face", 0, "faceIndex", "All")
attr.setReinforcementValue("face", 0, "actualCover", 0.03)
attr.setReinforcementValue("face", 0, "linkAllowance", 0.0)
attr.setReinforcementValue("face", 0, "allowableCrackWidth", 0.0)
attr.setReinforcementValue("face", 0, "nominalCover", -1.0)

add_rebar(attr, row=0, faceIndex=1, layerIndex=1, barsCount=3, start=-1, end=-1, gap=0.0, dia=0.025)
add_rebar(attr, row=1, faceIndex=1, layerIndex=2, barsCount=2, start=-1, end=-1, gap=0.025, dia=0.025)
add_rebar(attr, row=2, faceIndex=5, layerIndex=1, barsCount=10, start=-1, end=-1, gap=0.0, dia=0.025)


rebar_line_attr = db.createReinforcementLine("RnfLn2")
rebar_line_attr.setSegmentValue("Longitudinal", 0, "reinforcementSection", "RnfSct1")
rebar_line_attr.setSegmentValue("Longitudinal", 0, "stretch", True)

# Link the geometric attribute with the associated reinforcement
geom_section_attr.setReinforcement(rebar_line_attr)


# Materials

In order to use the prestress loading the concrete material requires additional environmental properties used in the prestress calculation. These are specified in the "Advanced Define" option in the material library.

In [ ]:
concrete_values = {"Material":"Concrete", "Region":"Europe", "Standard":"EN1992-1-1:2004/2014", "Grade":"fck = 35MPa Quartzite",
                   "Advanced define":1, "RH":70.0, "K1":1.0, "IntFac":0.5, "Aggregate type":0.0, "Cement type":1.0}

concrete_material_attr = db.createIsotropicMaterial("Concrete", Ec_shrt, 0.2, 2.54842, 10.0E-6)
concrete_material_attr.setDefinitionMenuID(1, None, True)
concrete_material_attr.setDescription("fck = 35MPa Quartzite | Concrete | EN1992-1-1:2004/2014")

for name, value in concrete_values.items():
    concrete_material_attr.createValue(name)
    concrete_material_attr.setValue(name, value)

# Values that must be defined with dimensionality
concrete_material_attr.createValue("fck", 0, 1, -2, 0, 0, 0, 0)
concrete_material_attr.setValue("fck", 35.0E3)

In [ ]:
rebar_values = {"Material":"Steel - Reinforcing bar", "Region":"Europe", "Standard":"EN1992-1-1:2004/2014", "Grade":"500B", "epu":0.05}

rebar_material_attr = db.createIsotropicMaterial("Rebar", 200.0E6, 0.3, 7.85, 12.0E-6)
rebar_material_attr.setDefinitionMenuID(1, None, True)
rebar_material_attr.setDescription("500B | Steel - Reinforcing bar | EN1992-1-1:2004/2014")

for name, value in rebar_values.items():
    rebar_material_attr.createValue(name)
    rebar_material_attr.setValue(name, value)

# Values that must be defined with dimensionality
rebar_material_attr.createValue("fy", 0, 1, -2, 0, 0, 0, 0)
rebar_material_attr.setValue("fy", 500.0e3)

In [ ]:
material_attr = db.createCompoundMaterial("Mat3").setReinforcedConcrete()
material_attr.addMaterial(concrete_material_attr, "S1")
material_attr.addMaterial(rebar_material_attr, "R1")
material_attr.setActiveMaterial(0)

material_attr.setDefinitionMenuID(214, None , True)
material_attr.setDescription("RC Material - EN1992")
material_attr.createValue("Design code")
material_attr.setValue("Design code", "EN1992")

In [ ]:
# Support attributes
fixedSupport = db.createSupportStructural("Fixed").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "F")
pinnedSupport = db.createSupportStructural("Pinned").setStructural("F", "R", "R", "F", "F", "F", "F", "F", "F")

#### Create the model geometry

In [ ]:
''' Create deck lines'''
deck_lines:list[IFLine] = []

cur_x = 0
for length in span_lengths:
    deck_lines.append(Helpers.create_line([cur_x, 0, 0], [cur_x + length, 0, 0]))
    # Increment the current x position
    cur_x += length


#### Assign attributes to the model geometry

In [ ]:
''' Assign the attributes to the deck_lines '''
# get the assignment object
assignment = lusas.assignment().setAllDefaults()
# Assign the mesh
mesh_attr.assignTo(deck_lines, assignment)
# Assign the geometry
geom_section_attr.assignTo(deck_lines, assignment)
# Assign the material
material_attr.assignTo(deck_lines, assignment)

''' Assign the supports to the points of the line '''
# Assign the fixed support to the first point
fixedSupport.assignTo(deck_lines[0].getStartPoint(), assignment)
# Assign the pinned support to the remaining points
pinnedSupport.assignTo([p.getEndPoint() for p in deck_lines], assignment)

In [ ]:
db.updateMesh()

# Loading

#### Create load cases and load attributes and assign the attributes to the model geometry

In [ ]:
db.getAnalysis("Analysis 1").setName("00 Base Analysis")

In [ ]:
# Get the automatically created loadcase in analysis 1 and add automatic gravity to it
# NOTE: getLoadset and setName function returns a reference to the IFLoadset baseclass and must be cast to IFLoadcase to access the addGravity function
gravity_loadcase = win32.CastTo(db.getLoadset("Loadcase 1", 0).setName("Gravity"), "IFLoadcase")
gravity_loadcase.addGravity(True)

In [ ]:
''' Create a beam load attribute '''
loadAttr = db.createLoadingBeamDistributed("UDL")
loadAttr.setBeamDistributed("Parametric", "Global", "beam")
loadAttr.addRow(0.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0)
# Assign the loading
loadAttr.assignTo(deck_lines, assignment)

# Create a loadcase for the Parapet loads to be applied to the entire deck surface
live_load_loadcases = []
for i in range(len(deck_lines)):
    live_load_loadcases.append(db.createLoadcase(f"Live Load {i+1}"))

# Assign live loads to each span
for i, loadcase in enumerate(live_load_loadcases):
    assign = lusas.assignment().setAllDefaults().setLoadset(loadcase)
    loadAttr.assignTo(deck_lines[i], assign)

# Prestressing

In [ ]:
# Create tendon profile. Utilities > Prestress > Tendon Profile
tendon_profile :IFTendonProfile = db.createTendonProfile("Tendon Profile")
tendon_profile.setLocal(True)

tendon_definition = tendon_profile.getDefn()
tendon_definition.deleteAllLines()

ZERO = 0.0
x = 0.0 # Current position of span start
# Set the tendon as a series of straight segments for simplicity. The lines must be continuous, so check the return values for errors
for i, length in enumerate(span_lengths):
    if i == 0:
        # First Span
        err1 = tendon_definition.addStraightV(x,              ZERO, ZERO, x + 0.4*length, ZERO, minz)
        err2 = tendon_definition.addStraightV(x + 0.4*length, ZERO, minz, x + length,     ZERO, maxz)
    elif i == len(span_lengths)-1:
        # Last span
        err1 = tendon_definition.addStraightV(x,              ZERO, maxz, x + 0.6*length, ZERO, minz)
        err2 = tendon_definition.addStraightV(x + 0.6*length, ZERO, minz, x + length,     ZERO, ZERO)
    else:
        # intermediate spans
        err1 = tendon_definition.addStraightV(x,              ZERO, maxz, x + 0.5*length, ZERO, minz)
        err2 = tendon_definition.addStraightV(x + 0.5*length, ZERO, minz, x + length,     ZERO, maxz)
    assert err1 == 0, f"Tendon definition failed span {i}, seg 1 - error {err1}"
    assert err2 == 0, f"Tendon definition failed span {i}, seg 2 - error {err2}"
    x+=length

print("Tendon Length = ", tendon_definition.getLength())


### Tendon Properties

There are two types of loss calculation for tendons in LUSAS. The first case carries out iterative analyses to calculate losses based analysed concrete stresses. The second is an approximate approach in which the stresses are assumed. Both cases will be considered and compared.

In [ ]:
# The approximate approaches requires additional specification of material properties and expected stresses
area = geom_section_attr.getValue("A")
Iyy = geom_section_attr.getValue("Iyy")
tendon_properties_dictionary_approx = dict(tendon_properties_dictionary, 
                                      **{"timeAfterT":time_at_end_of_service, "transfer":approx_stress_at_transfer, "creepE":phi, "conshr":Ec_shrt, "conlng":Ec_shrt, 
                                         "shkstn":0.4, "changeS":approx_change_in_stress_after_transfer,"conara":area, "conita":Iyy})

In [ ]:
# Create tendon properties. Utilities > Prestress > Tendon Properties

# First with the approximate settings
tendon_properties_approx_attr = db.createTendonProperties("Tendon Properties - Instantaneous - Approximate")
tendon_properties_approx_attr.setDesignCode("EN1992-1-1:2004 / 2014 Eurocode 2", False)
for name, value in tendon_properties_dictionary_approx.items():
    tendon_properties_approx_attr.setValue(name, value)

# Then with the analytical approach
tendon_properties_attr = db.createTendonProperties("Tendon Properties - Instantaneous - Analytical")
tendon_properties_attr.setDesignCode("EN1992-1-1:2004 / 2014 Eurocode 2", True)
for name, value in tendon_properties_dictionary.items():
    tendon_properties_attr.setValue(name, value)

In [ ]:
# Add long term losses
tendon_properties_dictionary['lossType'] = 1

# First with the approximate settings
tendon_properties_long_term_approx_attr = db.createTendonProperties("Tendon Properties - Long Term - Approximate")
tendon_properties_long_term_approx_attr.setDesignCode("EN1992-1-1:2004 / 2014 Eurocode 2", False)
for name, value in tendon_properties_dictionary_approx.items():
    tendon_properties_long_term_approx_attr.setValue(name, value)

# Then with the analytical approach
tendon_properties_long_term_attr = db.createTendonProperties("Tendon Properties - Long Term - Analytical")
tendon_properties_long_term_attr.setDesignCode("EN1992-1-1:2004 / 2014 Eurocode 2", True)
for name, value in tendon_properties_dictionary.items():
    tendon_properties_long_term_attr.setValue(name, value)

In [ ]:
age_attr = db.createAge("Age - 7 days")
age_attr.setAgeType("specified")
age_attr.setAgeAtActivation(7)
age_attr.setAgeAtShrinkage(3)
age_attr.assignTo(deck_lines)

In [ ]:
tendon_load_approx_attr = db.createLoadingTendon(f"Tendon Load {tendon_force:.1f}kN - Approx")
tendon_load_approx_attr.setForce(tendon_force)
tendon_load_approx_attr.setJackingEnd(1, 0.0, 5.0E-3)
tendon_load_approx_attr.unsetJackingEnd(2)
tendon_load_approx_attr.setProfile(tendon_profile)
tendon_load_approx_attr.setProperty(tendon_properties_approx_attr)

tendon_load_attr = db.createLoadingTendon(f"Tendon Load {tendon_force:.1f}kN - Analytical")
tendon_load_attr.setForce(tendon_force)
tendon_load_attr.setJackingEnd(1, 0.0, 5.0E-3)
tendon_load_attr.unsetJackingEnd(2)
tendon_load_attr.setProfile(tendon_profile)
tendon_load_attr.setProperty(tendon_properties_attr)

In [ ]:
tendon_load_long_term_approx_attr = db.createLoadingTendon(f"Tendon Load {tendon_force:.1f}kN - Long Term - Approx")
tendon_load_long_term_approx_attr.setForce(tendon_force)
tendon_load_long_term_approx_attr.setJackingEnd(1, 0.0, 5.0E-3)
tendon_load_long_term_approx_attr.unsetJackingEnd(2)
tendon_load_long_term_approx_attr.setProfile(tendon_profile)
tendon_load_long_term_approx_attr.setProperty(tendon_properties_long_term_approx_attr)

tendon_load_long_term_attr = db.createLoadingTendon(f"Tendon Load {tendon_force:.1f}kN - Long Term - Analytical")
tendon_load_long_term_attr.setForce(tendon_force)
tendon_load_long_term_attr.setJackingEnd(1, 0.0, 5.0E-3)
tendon_load_long_term_attr.unsetJackingEnd(2)
tendon_load_long_term_attr.setProfile(tendon_profile)
tendon_load_long_term_attr.setProperty(tendon_properties_long_term_attr)

Create a separate analysis for each approach

In [ ]:
prestress_analyses:list[IFAnalysisBaseClass] = []
prestress_loadcases:list[IFLoadcase] = []

for i, analysis in enumerate(["Approx", "Analytical"]):
    prestress_analysis = db.createAnalysisStructural(f"0{i+1} Prestress - {analysis}", False)
    # Create primary, secondary and effects without prestress - ignore all otherse
    prestress_analysis.setPrestressOptions(True, True, False, False, False, False, False, False, False, False, False, False, False, False, 1, 0.0)
    prestress_analyses.append(prestress_analysis)
    
    # Add loadcase for prestress loading
    for j, case in enumerate(["Instantaneous", "Long term"]):
        prestress_loadcase = db.createLoadcase(f"Prestress - {analysis} - {case}", prestress_analysis.getName())
        prestress_loadcase.addGravity(True) # Calculate the stresses in the concrete as a result of the prestress and gravity forces
        prestress_loadcases.append(prestress_loadcase)

        assign = lusas.assignment().setAllDefaults()
        assign.setMultipleId(1)
        assign.setLoadset(prestress_loadcase)
        assign.setLoadFactor(1.0)
        assign.setSearchAssignType("line")
        assign.setIncludedMoments("All")
        assign.setTendonProperties(False, 0.0)
        if i == 0:
            if j == 0:
                tendon_load_approx_attr.assignTo(deck_lines, assignment)
            else:
                tendon_load_long_term_approx_attr.assignTo(deck_lines, assignment)
        else:
            if j == 0:
                tendon_load_attr.assignTo(deck_lines, assignment)
            else:
                tendon_load_long_term_attr.assignTo(deck_lines, assignment)


In [ ]:
# The time at which the losses are to be calculated must be set on the loadcase for the analytical approach
prestress_loadcases[0].setLoadcaseAge(0.0)
prestress_loadcases[1].setLoadcaseAge(time_at_end_of_service)
prestress_loadcases[2].setLoadcaseAge(0.0)
prestress_loadcases[3].setLoadcaseAge(time_at_end_of_service)

#### Create design combinations

In [ ]:
# Smart Combination for permanent effects
combination_qp = db.createCombinationSmart("SLS QP")
combination_qp.addEntry(1.0, 0.0, prestress_loadcases[1].getID(), 2) # "Total effects" results of prestress loadcase (includes gravity)
for loadcase in live_load_loadcases:
    combination_qp.addEntry(0, 0.5, loadcase)

In [ ]:
# Smart Combination for service effects
combination_char = db.createCombinationSmart("SLS Char")
combination_char.addEntry(1.0, 0.0, prestress_loadcases[0].getID(), 2) # "Total effects" results of prestress loadcase (includes gravity)
for loadcase in live_load_loadcases:
    combination_char.addEntry(0, 1.0, loadcase)

In [ ]:
# Smart Combination for Ultimate effects
combination_uls = db.createCombinationSmart("ULS")
combination_uls.addEntry(1.0, 0.0, prestress_loadcases[0].getID(), 2) # "Total effects" results of prestress loadcase (includes gravity)
for loadcase in live_load_loadcases:
    combination_uls.addEntry(0, 1.5, loadcase)

In [ ]:
report = db.createReport("Report Approximate", "", "kN,m,t,s,C", 6, False)
chapter = report.addPrestressChapter("Prestress", -1)
chapter.setAssignmentIndexes([1])

report = db.createReport("Report Analytical", "", "kN,m,t,s,C", 6, False)
chapter = report.addPrestressChapter("Prestress", -1)
chapter.setAssignmentIndexes([2])


<H2>Solving the Analysis</H2>

In [ ]:
db.save()

# for analysis in db.getAnalyses():
#     analysis.solve(True)

# db.openAllResults(True, True)

In [ ]:
lusas.view().insertDiagramsLayer()
lusas.view().diagrams().setResultsTransformNone()
lusas.view().diagrams().setResults("Force/Moment - Thick 3D Beam", "My")
lusas.view().setFromAxis("+X+Z")
lusas.view().setScaledToFit(True)
lusas.view().attributes().visualiseAllTransparent("Geometric")


### Plot graphs of prestress forces

In [ ]:
# # Create a graph wizard to contain the graph settings
# graphWizardObj = db.createGraphWizard("Prestress Force vs Distance along beam")
# graphWizardObj.setAutoUpdate(1)
# graphWizardObj.setDisplayNow(1)
# graphWizardObj.setTitle("Force vs Distance along beam")
# graphWizardObj.setAxesLabels("Distance (m)", "Force (kN)")
# # 2No. assignments
# graphArray = graphWizardObj.createFromPrestress([1, 2])

# # Generate the actual graphs for display
# graphObj = lusas.newGraph()
# graphObj.setSourceAttrName("Force vs Distance along beam")
# graphObj.setTitle("Force vs Distance along beam")
# graphObj.setAxesLabels("Distance (m)", "Force (kN)")
# graphObj.autoMinX()
# graphObj.autoMaxX()
# graphObj.autoMinY()
# graphObj.autoMaxY()
# graphObj.setLinearX()
# graphObj.setLinearY()
# graphObj.showGrid()
# # 2No. assignments
# graphArray = graphObj.createFromPrestress([1, 2])

# if len(graphArray) == 4:

#     curve = graphObj.addCurve(graphArray[0], graphArray[1], 1.0, 1.0)
#     curve.showSymbols("DOT")
#     curve.setCurveColour(255, 0, 0)
#     curve = graphObj.addCurve(graphArray[2], graphArray[3], 1.0, 1.0)
#     curve.showSymbols("DOT")
#     curve.setCurveColour(0, 255, 0)

